In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
tf.keras.backend.set_floatx('float32')
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
    
from sklearn.datasets import load_sample_image

# Convolution - Faltung

Der Begriff der Faltung ist sehr mächtig. In der Bildverarbeitung wird er eingesetzt, um Filter auf die Bilddaten anzuwenden.


In [ ]:
china = load_sample_image("china.jpg")

fig, ax = plt.subplots(1, figsize=(8,8))
ax = plt.axes(xticks=[], yticks=[])
ax.imshow(china);

In [ ]:
print(f'Shape: {china.shape}')
china = tf.convert_to_tensor(china / 255.0, dtype=tf.float32);

Wir definieren uns eine Hilfsfunktion, die uns das Setup der Convolution abnimmt:

In [ ]:
def convolute_image(kernel, image):
    k = tf.convert_to_tensor(kernel)
    k = tf.tile(k[:, :, tf.newaxis, tf.newaxis], [1, 1, 3, 1])
    pointwise_filter = tf.eye(3, batch_shape=[1, 1])
    image = tf.nn.separable_conv2d(tf.expand_dims(image, 0), k, pointwise_filter,
                                   strides=[1, 1, 1, 1], padding='SAME')
    image = tf.squeeze(image)
    image = tf.math.minimum(tf.math.maximum(image, 0.), 1.)
    return image

Jetzt definieren wir zwei Filter: Diese hier sind geeignet, um linke Kanten bzw. obere Kanten zu verstärken.

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(20,8))
vertical = [[1.,0.,-1.], 
            [2.,0.,-2.], 
            [1.,0.,-1.]]

horizontal = [[1.,2.,1.], 
              [0.,0.,0.], 
              [-1.,-2.,-1.]]

axs[0].imshow(convolute_image(vertical, china))
axs[1].imshow(convolute_image(horizontal, china));

Zwei weitere Filter: Umrissfilter und Unschärfefilter.

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(20,8))
contour = [[0.,  0.,  0.,  0., 0.], 
           [0.,  0., -2.,  0., 0.], 
           [0., -2.,  8., -2., 0.], 
           [0.,  0., -2.,  0., 0.], 
           [0.,  0.,  0.,  0., 0.]]

# blur = [[0.04, 0.04, 0.04, 0.04, 0.04],
#         [0.04, 0.04, 0.04, 0.04, 0.04],
#         [0.04, 0.04, 0.04, 0.04, 0.04],
#         [0.04, 0.04, 0.04, 0.04, 0.04],
#         [0.04, 0.04, 0.04, 0.04, 0.04]]

blur = 0.04 * np.ones((5,5), dtype='float32') 

axs[0].imshow(convolute_image(contour, china))
axs[1].imshow(convolute_image(blur, china));

Warum ist beim verwendeten Unschärfefilter der Wert 0.04 sinnvoll? Was würde passieren, wenn wir einen größeren oder kleineren Wert nähmen?

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(20,8))
faktor = ???

axs[0].imshow(convolute_image(blur, china));
axs[1].imshow(convolute_image(faktor * 0.04 * np.ones((5,5), dtype='float32') , china));